# REPOSITORIO UNA

Geraldine Gianella Geronimo Oscanoa   20210836

#### TESIS PREGRADO DE LA ESCUELA PROFESIONAL INGENIERÍA ESTADÍSTICA E INFORMÁTICA

In [77]:
# Cargando librerías o módulos pertinentes
import requests
from bs4 import BeautifulSoup
import re
import itertools

In [78]:
# Creando una función que permite extraer todas las urls de las páginas que contienen
# las publicaciones encontradas en erespectivo repositorio
def all_pages(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene el url base de las páginas que contienen las publicaciones
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url base de las páginas que contienen las publicaciones
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["http://repositorio.unap.edu.pe/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de urls de cada una de las páginas 
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

In [79]:
# Creando una función que permite extraer todas las urls de la tabla de datos de las
# publicaciones de una determina página
def links_scrapy(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag padre de las publicaciones que contiene su respectivo url
    publication = soup.find_all("div", {"class":"artifact-title"})
    # Extraer los url de los tags hijos respectivos
    publication_soup = BeautifulSoup(str(publication), 'html.parser')
    a_tags = publication_soup.find_all('a') # find all anchor elements
    hrefs = ["http://repositorio.unap.edu.pe" + a_tag['href'] + "?show=full" for a_tag in a_tags]
    return hrefs

In [80]:
# Extracción de todas las urls de la tabla de datos de todas las
# publicaciones del repositorio
# NOTA: no se recomienda ejecutar esta celda, ya que toma demasiado tiempo obtener una salida
#every_page = all_pages("http://tesis.unap.edu.pe/browse?type=dateissued")
#publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page]))

In [81]:
# EJEMPLO DE EJECUCIÓN
# En este ejemplo solo se considera las dos primeras páginas del repositorio
every_page = all_pages("http://tesis.unap.edu.pe/handle/20.500.14082/857/browse?type=dateissued")
publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page[0:2]]))
publications_links

['http://repositorio.unap.edu.pe/handle/20.500.14082/5476?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/18709?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/818?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/355?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/349?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/19385?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/772?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/350?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/356?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/678?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/726?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/5781?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/310?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/747?show=full',
 'http://repositorio.unap.ed

In [82]:
import requests
import pandas as pd
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

#Creamos las listas donde guardaremos la información de las tesis
años = []
instituciones = []
titulos = []
autores = []
contribuidores = []
grados = []
resumenes = []
#Añadimos un contador que será de utilidad para colocar un dato vacio en caso la tesis no cuente con la información que se requiere 
n = 0

#
for i in range(len(publications_links)):
    respuesta = requests.get(publications_links[i], headers=headers)
    tabla_tesis = pd.read_html(respuesta.content, encoding = 'utf8')
    tabla_1 = tabla_tesis[0]

#Seleccionamos el dato de la fecha de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.date.issued':
            año=tabla_1.iloc[j,1]
            años.append(año[0:4])
            n = n + 1
    if n == 0:
        años.append(" ")

    else:
        n = 0


#Seleccionamos el dato de la institución de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.publisher':
            institucion=tabla_1.iloc[j,1]
            instituciones.append(institucion)
            n = n + 1
    if n == 0:
        instituciones.append(" ")

    else:
        n = 0


#Seleccionamos el dato del titulo de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):        
        if tabla_1.iloc[j,0] == 'dc.title':
            titulo=tabla_1.iloc[j,1]
            titulos.append(titulo)
            n = n + 1
    if n == 0:
            titulos.append(" ")

    else:
        n = 0

#Seleccionamos el dato del autor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.author':
            autor=tabla_1.iloc[j,1]
            autores.append(autor)
            n = n + 1
#En caso exista más de un autor, sólo seleccionamos el primero
            break
    if n == 0:
            autores.append(" ")

    else:
        n = 0 


#Seleccionamos el dato del grado de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.name':
            grado=tabla_1.iloc[j,1]
            grados.append(grado)
            n = n + 1
    if n == 0:
            grados.append(" ")

    else:
        n = 0


#Seleccionamos el dato del asesor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.advisor':
            contribuidor=tabla_1.iloc[j,1]
            contribuidores.append(contribuidor)
            n = n + 1
#En caso exista más de un asesor, sólo seleccionamos el primero
            break
    if n == 0:
            contribuidores.append(" ")
    else:
        n = 0


#Seleccionamos el dato del resumen de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.description.abstract':
            resumen=tabla_1.iloc[j,1]
            resumenes.append(resumen)
            n = n + 1
    if n == 0:
            resumenes.append(" ")
    else:
        n = 0



#print('Lista de años: ',años)
#print('Lista de instituciones: ',instituciones)
#print('Lista de titulos: ',titulos)
#print('Lista de autores: ',autores)
#print('Lista de contribuidores: ',contribuidores)
#print('Lista de grados: ',grados)
#print('Lista de resumenes: ',resumenes)

In [83]:
#Creamos un data frame donde se guardará todos los datos de las tesis de manera ordenada
df_pre = pd.DataFrame()

df_pre['instituciones'] = instituciones
df_pre['titulos'] = titulos
df_pre['autores'] = autores
df_pre['contribuidores'] = contribuidores
df_pre['resumenes'] = resumenes
df_pre['grados'] = grados
df_pre['años'] = años


#### TESIS POSGRADO DE LA ESCUELA PROFESIONAL INGENIERÍA ESTADÍSTICA E INFORMÁTICA

In [84]:
every_page = all_pages("http://tesis.unap.edu.pe/handle/20.500.14082/153/browse?type=dateissued")
publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page[0:2]]))
publications_links

['http://repositorio.unap.edu.pe/handle/20.500.14082/5476?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/18709?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/818?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/355?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/349?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/19385?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/772?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/350?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/356?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/678?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/726?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/5781?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/310?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/747?show=full',
 'http://repositorio.unap.ed

In [85]:
import requests
import pandas as pd
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

#Creamos las listas donde guardaremos la información de las tesis
años = []
instituciones = []
titulos = []
autores = []
contribuidores = []
grados = []
resumenes = []
#Añadimos un contador que será de utilidad para colocar un dato vacio en caso la tesis no cuente con la información que se requiere 
n = 0

#
for i in range(len(publications_links)):
    respuesta = requests.get(publications_links[i], headers=headers)
    tabla_tesis = pd.read_html(respuesta.content, encoding = 'utf8')
    tabla_1 = tabla_tesis[0]

#Seleccionamos el dato de la fecha de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.date.issued':
            año=tabla_1.iloc[j,1]
            años.append(año[0:4])
            n = n + 1
    if n == 0:
        años.append(" ")

    else:
        n = 0


#Seleccionamos el dato de la institución de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.publisher':
            institucion=tabla_1.iloc[j,1]
            instituciones.append(institucion)
            n = n + 1
    if n == 0:
        instituciones.append(" ")

    else:
        n = 0


#Seleccionamos el dato del titulo de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):        
        if tabla_1.iloc[j,0] == 'dc.title':
            titulo=tabla_1.iloc[j,1]
            titulos.append(titulo)
            n = n + 1
    if n == 0:
            titulos.append(" ")

    else:
        n = 0

#Seleccionamos el dato del autor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.author':
            autor=tabla_1.iloc[j,1]
            autores.append(autor)
            n = n + 1
#En caso exista más de un autor, sólo seleccionamos el primero
            break
    if n == 0:
            autores.append(" ")

    else:
        n = 0 


#Seleccionamos el dato del grado de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.name':
            grado=tabla_1.iloc[j,1]
            grados.append(grado)
            n = n + 1
    if n == 0:
            grados.append(" ")

    else:
        n = 0


#Seleccionamos el dato del asesor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.advisor':
            contribuidor=tabla_1.iloc[j,1]
            contribuidores.append(contribuidor)
            n = n + 1
#En caso exista más de un asesor, sólo seleccionamos el primero
            break
    if n == 0:
            contribuidores.append(" ")
    else:
        n = 0


#Seleccionamos el dato del resumen de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.description.abstract':
            resumen=tabla_1.iloc[j,1]
            resumenes.append(resumen)
            n = n + 1
    if n == 0:
            resumenes.append(" ")
    else:
        n = 0



#print('Lista de años: ',años)
#print('Lista de instituciones: ',instituciones)
#print('Lista de titulos: ',titulos)
#print('Lista de autores: ',autores)
#print('Lista de contribuidores: ',contribuidores)
#print('Lista de grados: ',grados)
#print('Lista de resumenes: ',resumenes)

In [86]:
#Creamos un data frame donde se guardará todos los datos de las tesis de manera ordenada
df_pos = pd.DataFrame()

df_pos['instituciones'] = instituciones
df_pos['titulos'] = titulos
df_pos['autores'] = autores
df_pos['contribuidores'] = contribuidores
df_pos['resumenes'] = resumenes
df_pos['grados'] = grados
df_pos['años'] = años


In [87]:
#Unimos los data framen de pregrado y posgrado para su importación
df = pd.DataFrame(df_pre.append(df_pos, 
                  ignore_index = True))
df.to_csv('UNA.csv', encoding='utf-8', index=False)

C:\Users\Geraldine\AppData\Local\Temp\ipykernel_34660\2114406576.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = pd.DataFrame(df_pre.append(df_pos,


In [88]:
df

,instituciones,titulos,autores,contribuidores,resumenes,grados,años
0,Universidad Nacional del Altiplano,Análisis y evaluación de resultados de gestión...,"Aliaga Ortega, Wenceslao","Cutipa Cari, Javier","El presente trabajo, se ha realizado en el dis...",Contador Público,1985
1,Universidad Nacional del Altiplano,Influencias de las actitudes paternales en la ...,"Ramos Torres, Lourdes Agueda","Monroy Gallegos, Samuel",La educación como proceso de enseñanza-aprendi...,"Licenciado en Educación, Especialidad de Lengu...",1992
2,Universidad Nacional del Altiplano,Respuesta de familias campesinas del distrito ...,"Supo Condori, Felipe","Ponce Arenas, Ricardo","El trabajo de investigación titulado ""Respuest...",Magíster Scientiae en Desarrollo Rural,1996
3,Universidad Nacional del Altiplano,Poblaciones de carabidos (coleoptera) en agroe...,"Loza Del Carpio, Alfredo Ludwig","Bravo, Rosario",El presente trabajo reporta los resultados sob...,Magíster Scientiae en Agricultura Andina,1999
4,Universidad Nacional del Altiplano,Niveles de consumo y evaluación sensorial de d...,"Gonzales Arestegui, Delicias Vilma","Zea Flores, Wilfredo E.","Esta Investigación de ""NIVELES DE CONSUMO Y EV...",Magíster Scientiae en Desarrollo Rural,1999
...,...,...,...,...,...,...,...
75,Universidad Nacional del Altiplano,Utilización de la lenteja de agua (Lemna sp) p...,"Cornejo Olarte, Dalmiro Aurelio","Apaza Campos, Rolando",El cianuro por ser un compuesto altamente tóxi...,Magíster Scientiae en Extracción de Metales Es...,2004
76,Universidad Nacional del Altiplano,Administración y gestión de las autoridades de...,"Vizcarra Herles, Nina Eleonor","Boza Condorena, Edwin Guido","El presente trabajo de investigación, contiene...",Magíster Scientiae en Contabilidad y Administr...,2004
77,Universidad Nacional del Altiplano,Calidad de servicios en laboratorios de mecáni...,"Machaca Condori, Héctor Raúl","Mariaca Flores, Luis",La presente investigación titulada CALIDAD DE ...,Magíster Scientiae en Contabilidad y Administr...,2004
78,Universidad Nacional del Altiplano,Proyectos de ahorro de energía: caso Universid...,"Arcos Salazar, Luis Alberto","Zegarra Martínez, Guilfredo",El uso racional de energía en todos los sector...,Magíster Scientiae en Economía con mención en ...,2004


In [89]:
df[['años','grados']].describe()

,años,grados
count,80,80
unique,9,18
top,2003,Magíster Scientiae en Informática
freq,20,14
